In [1]:
#All the imports for our code
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [2]:
#reading the data
path = '/content/drive/MyDrive/data1.csv'
df = pd.read_csv(path)
df.drop(['Unnamed: 0'], axis=1,inplace=True)
df.head()

,score,tweet
0,0,cute that bummer shoulda got david carr third day
1,0,upset cant updat facebook text might cri resul...
2,0,dive mani time ball manag save rest go bound
3,0,whole bodi feel itchi like fire
4,0,behav im mad cant see


In [3]:
df.dropna(inplace=True)#droping null rows


Making a Tfidf vectorizer

In [4]:
df0 = df.sample(frac=0.01,random_state=101)#Tdif vector of 10% of database at 10k features
X = df0.tweet.values
Y = df0.score.values/4

vector = TfidfVectorizer(max_features=1000,lowercase=False,stop_words='english')
X = vector.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Multi-Layer perceptron

In [5]:
clf = MLPClassifier(hidden_layer_sizes=(400),learning_rate_init=0.002, momentum=0.87,max_iter=500)#best performing model

In [6]:
clf.fit(X_train, y_train)

acc_train = accuracy_score(y_train, clf.predict(X_train))
acc_test = accuracy_score(y_test, clf.predict(X_test))

print("Accuracy on training set: "+str(acc_train))
print("Accuracy on testing set: "+str(acc_test))


Accuracy on training set: 0.9666980835689601
Accuracy on testing set: 0.7046811184417217


Using FS

Here, we used VarianceThreshold to filter features as we want only the best features and we also want to reduce the calculation cost.

As even at threshold = 0.00001 we have 18000-19000 featuers and they are only repeating only one or two times. It gets worse as we increase the sample size.

Our goal was to select best 5500 - 7500 features for the current sample.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X)

VarianceThreshold(threshold=0)

In [ ]:
constant_filter.get_support().sum()

23693

In [ ]:
Quasi_fil = VarianceThreshold(threshold=0.0000232)
Quasi_fil.fit(X)

VarianceThreshold(threshold=2.32e-05)

In [ ]:
Quasi_fil.get_support().sum()

5710

In [ ]:
filtered_x_prc = Quasi_fil.transform(X)

Now after doing all that we finally changed our values to an array. It is the most ram consuming part. From this we saw that sample size = 30000 - 50000 is the best.

In [ ]:
x_practice = filtered_x_prc.toarray()

In [ ]:
print(x_practice)
print(x_practice.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(31831, 5710)


Here we generated a list with index values as 'word_i' where i is 0,1,2,3..... array.shape[1]. These will be useful for futher making a dataframe.

In [ ]:
# for getting dictinory keys

ar = []

for i in range(0,1):
	ar.append([])
 
#print(len(ar))

for i in range(x_practice.shape[1]):
  ar[0].append('word_'+str(i))

print(len(ar[0]))

5710


We created a dataframe with our weight list and index_values = ar. We did to make further process easier to understand.

In [ ]:
dataframe = pd.DataFrame(x_practice,columns=ar)

In [ ]:
dataframe.head()

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,word_11,word_12,word_13,word_14,word_15,word_16,word_17,word_18,word_19,word_20,word_21,word_22,word_23,word_24,word_25,word_26,word_27,word_28,word_29,word_30,word_31,word_32,word_33,word_34,word_35,word_36,word_37,word_38,word_39,...,word_5670,word_5671,word_5672,word_5673,word_5674,word_5675,word_5676,word_5677,word_5678,word_5679,word_5680,word_5681,word_5682,word_5683,word_5684,word_5685,word_5686,word_5687,word_5688,word_5689,word_5690,word_5691,word_5692,word_5693,word_5694,word_5695,word_5696,word_5697,word_5698,word_5699,word_5700,word_5701,word_5702,word_5703,word_5704,word_5705,word_5706,word_5707,word_5708,word_5709
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dataframe.shape

(31831, 5710)

### Linear Discriminant Analysis

From here we will get one value as array.


In [ ]:
lda=  LinearDiscriminantAnalysis()
x_ld_prc = lda.fit(x_practice,Y)

In [ ]:
x_ld_prc = lda.transform(x_practice)

In [ ]:
print(x_ld_prc.shape)

(31831, 1)


Model after LDA and Feature selection

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_ld_prc, Y, test_size=0.2, random_state=42)

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(400),learning_rate_init=0.002, momentum=0.87,max_iter=500)
clf.fit(X_train, y_train)

model_Evaluate(clf)

Accuracy of model on training data : 81.93528118127553
Accuracy of model on testing data : 82.11088424689807 

